In [6]:
impor
import tensorflow as tf


full_numpy_bitmap_ambulance.npy   full_numpy_bitmap_mermaid.npy
full_numpy_bitmap_crocodile.npy   full_numpy_bitmap_raccoon.npy
full_numpy_bitmap_eye.npy         full_numpy_bitmap_rifle.npy
full_numpy_bitmap_flamingo.npy    full_numpy_bitmap_snail.npy
full_numpy_bitmap_harp.npy        full_numpy_bitmap_stethoscope.npy


In [9]:
DATA = '../../../../data/quickdraw'
FILE_TEMPLATE = f'{DATA}/full_numpy_bitmap_%s.py'
categories = ['ambulance', 'crocodile', 'eye', 'flamingo', 'harp', 
              'mermaid', 'raccoon', 'rifle', 'snail', 'stethoscope']
filenames = [FILE_TEMPLATE % x for x in categories]

In [11]:
BATCH = 128
DIM = 28
N_IMAGES = 100000 # reduce if mem issues
N_FILES = len(categories)
IMAGES_PER_CATEGORY = N_IMAGES // N_FILES
IMAGES_PER_CATEGORY

10000